<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/tutorials/3-Practical/FinRL_PaperTrading_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import sys, os

root_path = os.getcwd()
root_path = os.path.join(root_path, '..', '..')
print(os.path.abspath(root_path))
sys.path.append(root_path)
root_path = os.path.join(root_path, '..', 'ElegantRL')
print(os.path.abspath(root_path))
sys.path.append(root_path)

/Users/tianyang/projects/finrl/syn/FinRL
/Users/tianyang/projects/finrl/syn/ElegantRL


In [13]:
from finrl.train import train, load_df
from finrl.test import test
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.plot import backtest_stats, get_daily_return, get_baseline # backtest_plot
from common import *
import datetime as dt
from finrl.plot import *
import pandas as pd

In [14]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)
candle_time_interval = '5Min'  # '1Min'

env = StockTradingEnv

ERL_PARAMS = {"learning_rate": 3e-6, "batch_size": 2048, "gamma": 0.985,
              "seed": 312, "net_dimension": 512, "target_step": 5000, "eval_gap": 30,
              "eval_times": 1}

train_start_date = '2022-6-11'
train_end_date = '2022-8-11'
test_start_date = '2022-8-11'
test_end_date = '2022-9-1'
baseline_ticker = 'AXP'

model_name = 'ppo'
MODEL_IDX = f'{model_name}_{train_start_date}_{train_end_date}'


In [15]:
def backtest_plot(
        account_value,
        baseline_df,
        value_col_name="account_value",
):
    df = deepcopy(account_value)
    # print('date', len(df))
    # print(type(df["date"][0]), df["date"][0])
    df["date"] = pd.to_datetime(df["date"])
    # df["date"] = pd.Timestamp(df["date"]).tz_localize("America/New_York")
    # df["date"] = df["date"].tz_localize("America/New_York")
    test_returns = get_daily_return(df, value_col_name=value_col_name)

    baseline_df["date"] = pd.to_datetime(baseline_df["date"], format="%Y-%m-%d")
    baseline_df = pd.merge(df[["date"]], baseline_df, how="left", on="date")
    baseline_df = baseline_df.fillna(method="ffill").fillna(method="bfill")
    baseline_returns = get_daily_return(baseline_df, value_col_name="close")

    with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(
            returns=test_returns, benchmark_rets=baseline_returns, set_context=False
        )



In [23]:
# train(start_date=train_start_date,
#       end_date=train_end_date,
#       ticker_list=ticker_list,
#       data_source='alpaca',
#       time_interval=candle_time_interval,
#       technical_indicator_list=INDICATORS,
#       drl_lib='elegantrl',
#       #       drl_lib='rllib',
#       #       drl_lib='stable_baselines3',
#       env=env,
#       model_name=model_name,
#       API_KEY=API_KEY,
#       API_SECRET=API_SECRET,
#       API_BASE_URL=API_BASE_URL,
#       erl_params=ERL_PARAMS,
#       cwd=f'./papertrading_erl/{MODEL_IDX}',  # current_working_dir
#       break_step=1e7)

account_value = test(start_date=test_start_date,
                     end_date=test_end_date,
                     ticker_list=ticker_list,
                     data_source='alpaca',
                     time_interval=candle_time_interval,
                     technical_indicator_list=INDICATORS,
                     drl_lib='elegantrl',
                     env=env,
                     model_name='ppo',
                     API_KEY=API_KEY,
                     API_SECRET=API_SECRET,
                     API_BASE_URL=API_BASE_URL,
                     #       erl_params=ERL_PARAMS,
                     cwd=f'./papertrading_erl/{MODEL_IDX}',  # current_working_dir
                     if_plot=True,  # to return a dataframe for backtest_plot
                     break_step=1e7)
print("============== account_value ===========")
print(account_value)



Alpaca successfully connected
Load data from /Users/tianyang/projects/finrl/syn/FinRL/tutorials/3-Practical/../../finrl/../data/alpaca_2022-6-11_2022-9-1.pkl
The data looks like: 
 <bound method DataFrame.to_string of                        timestamp     open      high       low     close  \
479700 2022-08-11 09:30:00-04:00  170.060  170.3000  169.9900  170.1600   
479701 2022-08-11 09:30:00-04:00  251.830  253.3600  251.8000  253.1200   
479702 2022-08-11 09:30:00-04:00  164.360  164.6400  164.0600  164.5850   
479703 2022-08-11 09:30:00-04:00  170.500  171.3999  170.2300  171.3400   
479704 2022-08-11 09:30:00-04:00  192.830  193.3800  192.6200  193.3800   
479705 2022-08-11 09:30:00-04:00  191.960  192.5000  190.1300  190.2700   
479706 2022-08-11 09:30:00-04:00   46.185   46.1850   45.9600   46.0000   
479707 2022-08-11 09:30:00-04:00  158.120  158.5400  158.0600  158.3950   
479708 2022-08-11 09:30:00-04:00  122.200  122.9100  122.0000  122.2600   
479709 2022-08-11 09:30:00-04:00

Successfully transformed into array
Failed loading models from ./papertrading_erl/ppo_2022-6-11_2022-8-11.
Test Finished!
episode_return 0.9699103375355531
============== account_value ===========
          date  account_value
0   2022-08-11   99507.426484
1   2022-08-12  100309.104974
2   2022-08-15  100629.582170
3   2022-08-16  101043.506666
4   2022-08-17  100569.530198
5   2022-08-18  100549.319613
6   2022-08-19  100318.049536
7   2022-08-22   98881.057550
8   2022-08-23   99084.821589
9   2022-08-24   99386.867865
10  2022-08-25  100018.255653
11  2022-08-26   98287.436993
12  2022-08-29   98152.518056
13  2022-08-30   97266.947815
14  2022-08-31   96991.033754


In [21]:

# baseline stats
print("==============Get Baseline Stats===========")
# baseline_df_dji = get_baseline(
#     ticker="^DJI",
#     start=test_start_date,
#     end=test_end_date)

baseline_df = load_df(test_start_date, test_end_date)
baseline_df = baseline_df[baseline_df['tic'] == baseline_ticker]
baseline_df = baseline_df.loc[baseline_df['timestamp'].dt.time == dt.time(15, 59)]
baseline_df['date'] = baseline_df['timestamp'].dt.date
stats = backtest_stats(baseline_df, value_col_name='close')
print(stats)



NameError: name 'account_value' is not defined

In [ ]:
print("==============Compare to Baseline===========")
backtest_plot(account_value, baseline_df)